# 🏈 Sleeper Fantasy League - Master Control Panel

Complete control over your Sleeper fantasy league data.

**Features:**
- 🗑️ Database management (view, clear, reset)
- 📊 Fetch league data (teams, matchups, standings)
- 👥 User and roster management
- 🏈 NFL player data and schedules
- 📈 Player stats and performance tracking
- 🔍 Advanced queries and analysis


## 📦 Setup & Configuration


In [2]:
import sqlite3
import json
from datetime import datetime
from database_league import LeagueDB
from scraper_sleeper_league import SleeperLeagueScraper
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# ==================== CONFIGURATION ====================
# Update these with your league information

SLEEPER_USERNAME = os.getenv("SLEEPER_USERNAME")  # Your Sleeper username from .env
LEAGUE_ID = os.getenv("LEAGUE_ID")  # League ID from .env
SEASON = "2025"
CURRENT_WEEK = 9  # Update this weekly

print("✓ Imports successful")
print(f"Configuration: Season {SEASON}, Week {CURRENT_WEEK}")
print(f"Sleeper Username: {SLEEPER_USERNAME}")
print(f"League ID: {LEAGUE_ID if LEAGUE_ID else 'Not set - run Step 1 to find your leagues'}")


✓ Imports successful
Configuration: Season 2025, Week 9
Sleeper Username: sfaizi24
League ID: 1226433368405585920


## 🔍 Step 1: Find Your Leagues

Run this to discover your league IDs


In [3]:
def find_my_leagues():
    """Find all leagues for your username."""
    with SleeperLeagueScraper() as scraper:
        # Get user
        user = scraper.get_user(SLEEPER_USERNAME)
        
        if not user:
            print(f"❌ User '{SLEEPER_USERNAME}' not found!")
            print("Please update SLEEPER_USERNAME in the configuration cell above.")
            return None
        
        user_id = user['user_id']
        print(f"\n👤 User: {user.get('display_name', SLEEPER_USERNAME)}")
        print(f"User ID: {user_id}\n")
        
        # Get leagues
        leagues = scraper.get_user_leagues(user_id, SEASON)
        
        if not leagues:
            print(f"❌ No leagues found for {SEASON}")
            return None
        
        print(f"\n{'='*70}")
        print(f"YOUR LEAGUES ({SEASON})")
        print(f"{'='*70}\n")
        
        for i, league in enumerate(leagues, 1):
            print(f"{i}. {league['name']}")
            print(f"   League ID: {league['league_id']}")
            print(f"   Total Rosters: {league.get('total_rosters', 'N/A')}")
            print(f"   Status: {league.get('status', 'N/A')}")
            print()
        
        print(f"{'='*70}")
        print("\n📝 Copy a League ID and paste it into the LEAGUE_ID configuration above.")
        
        return leagues

# Run this to find your leagues
my_leagues = find_my_leagues()


Fetching user: sfaizi24...
  ✓ Found user: sfaizi24 (ID: 1005347233318940672)

👤 User: sfaizi24
User ID: 1005347233318940672

Fetching leagues for user 1005347233318940672 in 2025...
  ✓ Found 4 leagues

YOUR LEAGUES (2025)

1. Redeem Team Association
   League ID: 1269052128316362752
   Total Rosters: 10
   Status: in_season

2. Abbas Khandaan
   League ID: 1266707166694223872
   Total Rosters: 10
   Status: in_season

3. Traveling Fantasy Circus 🎪
   League ID: 1226433368405585920
   Total Rosters: 12
   Status: in_season

4. Dynasty TNPunu League
   League ID: 1204901305273831424
   Total Rosters: 10
   Status: in_season


📝 Copy a League ID and paste it into the LEAGUE_ID configuration above.


## 📊 Database Status & Overview


In [1]:
def show_database_status():
    """Display current database status."""
    with LeagueDB() as db:
        print(f"\n{'='*70}")
        print(f"LEAGUE DATABASE STATUS")
        print(f"{'='*70}\n")
        
        # Leagues
        leagues = db.get_all_leagues()
        print(f"📊 Leagues: {len(leagues)}")
        for league in leagues:
            print(f"   - {league['name']} ({league['season']})")
        
        # Users
        if LEAGUE_ID:
            users = db.get_users(LEAGUE_ID)
            print(f"\n👥 Users in current league: {len(users)}")
            for user in users[:10]:  # Show first 10
                print(f"   - {user.get('display_name', user.get('username', 'Unknown'))}")
        else:
            all_users = db.get_users()
            print(f"\n👥 Total Users: {len(all_users)}")
        
        # Rosters
        if LEAGUE_ID:
            rosters = db.get_rosters(LEAGUE_ID)
            print(f"\n🏆 Rosters/Teams: {len(rosters)}")
            for roster in sorted(rosters, key=lambda x: x.get('wins', 0), reverse=True)[:5]:
                team_name = roster.get('team_name') or f"Team {roster['roster_id']}"
                owner = roster.get('display_name', 'Unknown')
                record = f"{roster.get('wins', 0)}-{roster.get('losses', 0)}"
                fpts = roster.get('fpts', 0)
                print(f"   {record:6} | {fpts:7.1f} pts | {team_name} ({owner})")
        
        # Matchups
        if LEAGUE_ID:
            matchups = db.get_matchups(LEAGUE_ID)
            weeks_with_matchups = len(set(m['week'] for m in matchups)) if matchups else 0
            print(f"\n🎮 Matchups: {len(matchups)} across {weeks_with_matchups} weeks")
        
        # NFL Players
        all_players = db.get_nfl_players()
        print(f"\n🏈 NFL Players: {len(all_players)}")
        
        # Position breakdown
        positions = {}
        for p in all_players:
            pos = p.get('position', 'Unknown')
            positions[pos] = positions.get(pos, 0) + 1
        
        if positions:
            print("   By Position:")
            for pos in ['QB', 'RB', 'WR', 'TE', 'K', 'DEF']:
                if pos in positions:
                    print(f"      {pos}: {positions[pos]}")
        
        # Player Stats
        all_stats = db.get_player_stats()
        if all_stats:
            weeks_with_stats = len(set(s['week'] for s in all_stats))
            print(f"\n📈 Player Stats: {len(all_stats)} records across {weeks_with_stats} weeks")
        else:
            print(f"\n📈 Player Stats: None loaded")
        
        # Schedules
        schedules = db.get_schedules(season=SEASON)
        if schedules:
            teams_count = len(set(s['team'] for s in schedules))
            print(f"\n📅 NFL Schedule: {len(schedules)} records for {teams_count} teams")
            
            # Show bye weeks
            bye_weeks = db.get_bye_weeks(SEASON)
            if bye_weeks:
                print(f"   Bye weeks loaded: {len(bye_weeks)} teams")
        else:
            print(f"\n📅 NFL Schedule: Not loaded")
        
        print(f"\n{'='*70}\n")

#show_database_status()


## 🚀 Step 2: Initial Data Load

Run this once to populate your database with league data


In [6]:
def initial_data_load():
    """Perform initial data load for the league."""
    if not LEAGUE_ID:
        print("❌ Please set LEAGUE_ID in the configuration cell first!")
        return
    
    print(f"\n{'='*70}")
    print(f"INITIAL DATA LOAD")
    print(f"{'='*70}\n")
    print(f"This will load:")
    print(f"  - League information")
    print(f"  - All users/owners")
    print(f"  - All rosters/teams")
    print(f"  - Matchups for weeks 1-{CURRENT_WEEK}")
    print(f"  - NFL player database")
    print(f"  - Player stats for weeks 1-{CURRENT_WEEK}")
    print(f"  - NFL schedule with bye weeks")
    print(f"\nThis may take 2-3 minutes...\n")
    
    with SleeperLeagueScraper() as scraper:
        # 1. Load league data
        print("📊 Loading league data...")
        scraper.save_league_data(
            league_id=LEAGUE_ID,
            weeks=list(range(1, CURRENT_WEEK + 1)),
            include_transactions=True
        )
        
        # 2. Load NFL players
        print("\n🏈 Loading NFL players...")
        scraper.save_nfl_players()
        
        # 3. Load player stats
        print(f"\n📈 Loading player stats (weeks 1-{CURRENT_WEEK})...")
        scraper.save_player_stats(SEASON, 1, CURRENT_WEEK)
        
        # 4. Load NFL schedule
        print("\n📅 Loading NFL schedule...")
        scraper.save_nfl_schedule(SEASON)
    
    print(f"\n{'='*70}")
    print(f"✅ INITIAL DATA LOAD COMPLETE!")
    print(f"{'='*70}\n")

# Uncomment to run:
initial_data_load()
show_database_status()



INITIAL DATA LOAD

This will load:
  - League information
  - All users/owners
  - All rosters/teams
  - Matchups for weeks 1-9
  - NFL player database
  - Player stats for weeks 1-9
  - NFL schedule with bye weeks

This may take 2-3 minutes...

📊 Loading league data...

FETCHING COMPLETE LEAGUE DATA

Fetching league details: 1226433368405585920...
  ✓ League: Traveling Fantasy Circus 🎪 (2025)
Fetching users for league 1226433368405585920...
  ✓ Found 12 users/owners
Fetching rosters for league 1226433368405585920...
  ✓ Found 12 rosters/teams

Fetching matchups for weeks: [1, 2, 3, 4, 5, 6, 7, 8, 9]
Fetching matchups for week 1...
  ✓ Found 12 team matchups (6 games)
Fetching matchups for week 2...
  ✓ Found 12 team matchups (6 games)
Fetching matchups for week 3...
  ✓ Found 12 team matchups (6 games)
Fetching matchups for week 4...
  ✓ Found 12 team matchups (6 games)
Fetching matchups for week 5...
  ✓ Found 12 team matchups (6 games)
Fetching matchups for week 6...
  ✓ Found 12 t

## 🔄 Weekly Update

Run this weekly to update matchups and stats


In [7]:
def weekly_update(week=None):
    """Update league data for current/specified week."""
    if not LEAGUE_ID:
        print("❌ Please set LEAGUE_ID in the configuration cell first!")
        return
    
    if week is None:
        week = CURRENT_WEEK
    
    print(f"\n{'='*70}")
    print(f"WEEKLY UPDATE - Week {week}")
    print(f"{'='*70}\n")
    
    with SleeperLeagueScraper() as scraper:
        # Update league data (rosters may have changed)
        print("📊 Updating league data...")
        scraper.save_league_data(
            league_id=LEAGUE_ID,
            weeks=[week],
            include_transactions=True
        )
        
        # Update player stats for this week
        print(f"\n📈 Updating player stats for week {week}...")
        scraper.save_player_stats(SEASON, week, week)
        
        # Refresh NFL players (injuries, roster changes)
        print("\n🏈 Refreshing NFL player data...")
        scraper.save_nfl_players()
    
    print(f"\n{'='*70}")
    print(f"✅ WEEKLY UPDATE COMPLETE!")
    print(f"{'='*70}\n")

# Uncomment to run:
weekly_update()
show_database_status()



WEEKLY UPDATE - Week 9

📊 Updating league data...

FETCHING COMPLETE LEAGUE DATA

Fetching league details: 1226433368405585920...
  ✓ League: Traveling Fantasy Circus 🎪 (2025)
Fetching users for league 1226433368405585920...
  ✓ Found 12 users/owners
Fetching rosters for league 1226433368405585920...
  ✓ Found 12 rosters/teams

Fetching matchups for weeks: [9]
Fetching matchups for week 9...
  ✓ Found 12 team matchups (6 games)

Fetching transactions for weeks: [9]
Fetching transactions for week 9...
  ✓ Found 0 transactions

DATA FETCH COMPLETE
League: Traveling Fantasy Circus 🎪
Users: 12
Rosters: 12
Matchups: 12 across 1 weeks
Transactions: 0


💾 Saving to database...
  ✓ Saved league: Traveling Fantasy Circus 🎪
  ✓ Saved 12 users
  ✓ Saved 12 rosters
  ✓ Saved 12 matchups across 1 weeks

✅ League data saved successfully!


📈 Updating player stats for week 9...

FETCHING PLAYER STATS

Fetching player stats for 2025 Week 9...
  ✓ Loaded stats for 0 players

PLAYER STATS COMPLETE
Week

## 📈 Analysis & Queries

### View League Standings


In [5]:
def show_standings():
    """Display league standings."""
    if not LEAGUE_ID:
        print("❌ Please set LEAGUE_ID first!")
        return
    
    with LeagueDB() as db:
        rosters = db.get_rosters(LEAGUE_ID)
        
        print(f"\n{'='*70}")
        print(f"LEAGUE STANDINGS")
        print(f"{'='*70}\n")
        print(f"{'Rank':<6} {'Record':<10} {'Points For':<12} {'Team (Owner)'}")
        print("-" * 70)
        
        for i, roster in enumerate(rosters, 1):
            team_name = roster.get('team_name') or f"Team {roster['roster_id']}"
            owner = roster.get('display_name', 'Unknown')
            record = f"{roster.get('wins', 0)}-{roster.get('losses', 0)}-{roster.get('ties', 0)}"
            fpts = roster.get('fpts', 0) + roster.get('fpts_decimal', 0)
            
            print(f"{i:<6} {record:<10} {fpts:<12.2f} {team_name} ({owner})")
        
        print(f"\n{'='*70}\n")

show_standings()



LEAGUE STANDINGS

Rank   Record     Points For   Team (Owner)
----------------------------------------------------------------------
1      7-1-0      1170.00      Team 8 (sahirsyed30)
2      7-1-0      1068.00      Team 1 (xavierking4)
3      5-3-0      976.00       Team 12 (sfaizi24)
4      4-4-0      1020.00      Team 3 (amir812)
5      4-4-0      1019.00      Team 2 (asadrafique)
6      4-4-0      1021.00      Team 6 (Jibraan)
7      4-4-0      1014.00      Team 7 (mehdidrissi)
8      4-4-0      935.00       Team 5 (TBK41)
9      4-4-0      969.00       Team 10 (monkeyman966699696)
10     3-5-0      965.00       Team 11 (Ammady)
11     1-7-0      818.00       Team 4 (umarrahman30)
12     1-7-0      860.00       Team 9 (Bilal879)




### View Matchups for Specific Week


In [9]:
def show_matchups(week=None):
    """Display matchups for a specific week."""
    if not LEAGUE_ID:
        print("❌ Please set LEAGUE_ID first!")
        return
    
    if week is None:
        week = CURRENT_WEEK
    
    with LeagueDB() as db:
        matchups = db.get_matchups(LEAGUE_ID, week)
        
        if not matchups:
            print(f"No matchups found for week {week}")
            return
        
        print(f"\n{'='*70}")
        print(f"MATCHUPS - Week {week}")
        print(f"{'='*70}\n")
        
        # Group by matchup_id
        matchup_groups = {}
        for m in matchups:
            mid = m.get('matchup_id_number')
            if mid not in matchup_groups:
                matchup_groups[mid] = []
            matchup_groups[mid].append(m)
        
        # Display each matchup
        for mid in sorted(matchup_groups.keys()):
            teams = matchup_groups[mid]
            
            if len(teams) == 2:
                team1, team2 = teams
                
                name1 = team1.get('team_name') or team1.get('display_name') or f"Team {team1['roster_id']}"
                name2 = team2.get('team_name') or team2.get('display_name') or f"Team {team2['roster_id']}"
                
                pts1 = team1.get('points', 0)
                pts2 = team2.get('points', 0)
                
                winner = "🏆" if pts1 > pts2 else "  "
                winner2 = "🏆" if pts2 > pts1 else "  "
                
                print(f"Matchup {mid}:")
                print(f"  {winner} {name1:<30} {pts1:>6.2f}")
                print(f"  {winner2} {name2:<30} {pts2:>6.2f}")
                print()
        
        print(f"{'='*70}\n")

# Show current week matchups
show_matchups()



MATCHUPS - Week 9

Matchup 1:
     TBK41                            0.00
     mehdidrissi                      0.00

Matchup 2:
     umarrahman30                     0.00
     Bilal879                         0.00

Matchup 3:
     sahirsyed30                      0.00
     monkeyman966699696               0.00

Matchup 4:
     xavierking4                      0.00
     Ammady                           0.00

Matchup 5:
     amir812                          0.00
     Jibraan                          0.00

Matchup 6:
     asadrafique                      0.00
     sfaizi24                         0.00




### View Team Roster


In [11]:
def show_team_roster(roster_id):
    """Display a team's roster."""
    if not LEAGUE_ID:
        print("❌ Please set LEAGUE_ID first!")
        return
    
    with LeagueDB() as db:
        roster = db.get_roster(roster_id, LEAGUE_ID)
        
        if not roster:
            print(f"Roster {roster_id} not found")
            return
        
        team_name = roster.get('team_name') or f"Team {roster_id}"
        owner = roster.get('display_name', 'Unknown')
        
        print(f"\n{'='*70}")
        print(f"{team_name} - {owner}")
        print(f"{'='*70}\n")
        print(f"Record: {roster.get('wins', 0)}-{roster.get('losses', 0)}-{roster.get('ties', 0)}")
        print(f"Points For: {roster.get('fpts', 0):.2f}")
        print(f"Points Against: {roster.get('fpts_against', 0):.2f}")
        print(f"Waiver Position: {roster.get('waiver_position', 'N/A')}")
        
        # Parse players
        import ast
        try:
            players_str = roster.get('players', '[]')
            if isinstance(players_str, str):
                player_ids = ast.literal_eval(players_str)
            else:
                player_ids = players_str
            
            if player_ids:
                print(f"\n📋 Roster ({len(player_ids)} players):")
                print("-" * 70)
                
                for player_id in player_ids[:20]:  # Show first 20
                    player = db.get_nfl_player(player_id)
                    if player:
                        name = player.get('full_name') or 'Unknown'
                        pos = player.get('position') or '??'
                        team = player.get('team') or 'FA'
                        status = player.get('injury_status') or ''
                        status_str = f" ({status})" if status else ""
                        print(f"  {pos:<3} {name:<25} {team:<4}{status_str}")
                
                if len(player_ids) > 20:
                    print(f"  ... and {len(player_ids) - 20} more")
        except Exception as e:
            print(f"\n⚠ Could not parse roster: {e}")
        
        print(f"\n{'='*70}\n")

# Show a specific team's roster (change the number)
show_team_roster(12)



Team 12 - sfaizi24

Record: 5-3-0
Points For: 906.00
Points Against: 815.00
Waiver Position: 12

📋 Roster (15 players):
----------------------------------------------------------------------
  RB  Isaiah Davis              NYJ 
  TE  Tyler Warren              IND 
  WR  Luther Burden             CHI  (Out)
  WR  Travis Hunter             JAX 
  RB  Derrick Henry             BAL 
  QB  Matthew Stafford          LAR 
  WR  Kendrick Bourne           SF  
  RB  Tony Pollard              TEN 
  QB  Justin Herbert            LAC 
  WR  Tee Higgins               CIN 
  RB  Chuba Hubbard             CAR 
  K   Cameron Dicker            LAC 
  WR  Puka Nacua                LAR  (Out)
  WR  Quentin Johnston          LAC 
  DEF ATL Defense               ATL 




### Check Bye Weeks


In [12]:
def show_bye_weeks(week=None):
    """Display bye weeks."""
    with LeagueDB() as db:
        if week:
            schedules = db.get_schedules(season=SEASON, week=week, is_bye=True)
            teams_on_bye = [s['team'] for s in schedules]
            
            print(f"\n{'='*70}")
            print(f"BYE WEEKS - Week {week}")
            print(f"{'='*70}\n")
            
            if teams_on_bye:
                print(f"Teams on bye: {', '.join(sorted(teams_on_bye))}")
            else:
                print("No teams on bye this week")
        else:
            bye_weeks = db.get_bye_weeks(SEASON)
            
            print(f"\n{'='*70}")
            print(f"ALL BYE WEEKS - {SEASON}")
            print(f"{'='*70}\n")
            
            # Group by week
            weeks_dict = {}
            for team, week_num in bye_weeks.items():
                if week_num not in weeks_dict:
                    weeks_dict[week_num] = []
                weeks_dict[week_num].append(team)
            
            for week_num in sorted(weeks_dict.keys()):
                teams = sorted(weeks_dict[week_num])
                print(f"Week {week_num:2}: {', '.join(teams)}")
        
        print(f"\n{'='*70}\n")

# Show all bye weeks
show_bye_weeks()

# Or check a specific week
# show_bye_weeks(week=10)



ALL BYE WEEKS - 2025

Week  5: ATL, CHI, GB, PIT
Week  6: HOU, MIN
Week  7: BAL, BUF
Week  8: ARI, DET, JAX, LAR, LV, SEA
Week  9: CLE, NYJ, PHI, TB
Week 10: CIN, DAL, KC, TEN
Week 11: IND, NO
Week 12: DEN, LAC, MIA, WAS
Week 14: CAR, NE, NYG, SF




### Player Performance Lookup


In [13]:
def show_player_stats(player_name):
    """Show stats for a specific player."""
    with LeagueDB() as db:
        # Find player
        all_players = db.get_nfl_players()
        matching_players = [p for p in all_players 
                          if player_name.lower() in (p.get('full_name') or '').lower()]
        
        if not matching_players:
            print(f"No players found matching '{player_name}'")
            return
        
        if len(matching_players) > 1:
            print(f"Found {len(matching_players)} matching players:")
            for p in matching_players[:10]:
                print(f"  - {p.get('full_name') or 'Unknown'} ({p.get('position')}, {p.get('team', 'FA')})")
            print("\nPlease be more specific")
            return
        
        player = matching_players[0]
        player_id = player['player_id']
        
        print(f"\n{'='*70}")
        print(f"{player.get('full_name') or 'Unknown'} - {player.get('position')} - {player.get('team', 'FA')}")
        print(f"{'='*70}\n")
        
        # Player info
        print(f"Height: {player.get('height', 'N/A')}")
        print(f"Weight: {player.get('weight', 'N/A')}")
        print(f"College: {player.get('college', 'N/A')}")
        print(f"Years Exp: {player.get('years_exp', 'N/A')}")
        print(f"Status: {player.get('status', 'N/A')}")
        
        injury = player.get('injury_status')
        if injury:
            print(f"Injury: {injury} - {player.get('injury_body_part', 'N/A')}")
        
        # Get stats
        stats = db.get_player_stats(player_id=player_id, season=SEASON)
        
        if stats:
            print(f"\n📈 {SEASON} Stats:")
            print("-" * 70)
            print(f"{'Week':<6} {'Team':<6} {'Opp':<6} {'PPR Pts':<10} {'Pass':<12} {'Rush':<12} {'Rec'}")
            print("-" * 70)
            
            total_pts = 0
            for stat in stats:
                week = stat.get('week') or 0
                team = stat.get('team') or 'N/A'
                opp = stat.get('opponent') or 'N/A'
                pts = stat.get('pts_ppr') or 0
                total_pts += pts
                
                # Position-specific stats
                pass_stat = f"{stat.get('pass_cmp') or 0}/{stat.get('pass_att') or 0}"
                rush_stat = f"{stat.get('rush_att') or 0} car"
                rec_stat = f"{stat.get('rec') or 0} rec"
                
                print(f"{week:<6} {team:<6} {opp:<6} {pts:<10.2f} {pass_stat:<12} {rush_stat:<12} {rec_stat}")
            
            avg_pts = total_pts / len(stats) if stats else 0
            print("-" * 70)
            print(f"Total: {total_pts:.2f} pts | Average: {avg_pts:.2f} pts/game")
        else:
            print(f"\nNo stats found for {SEASON}")
        
        print(f"\n{'='*70}\n")

# Look up a player
show_player_stats("Bijan Robinson")



Bijan Robinson - RB - ATL

Height: 71
Weight: 215
College: Texas
Years Exp: 2
Status: Active

📈 2025 Stats:
----------------------------------------------------------------------
Week   Team   Opp    PPR Pts    Pass         Rush         Rec
----------------------------------------------------------------------
8      N/A    N/A    5.80       0/0          9 car        3 rec
7      N/A    N/A    21.20      0/0          14 car       6 rec
6      N/A    N/A    35.80      0/0          19 car       6 rec
4      N/A    N/A    28.10      0/0          17 car       4 rec
3      N/A    N/A    16.10      0/0          13 car       5 rec
2      N/A    N/A    19.80      0/0          22 car       3 rec
1      N/A    N/A    24.40      0/0          12 car       6 rec
----------------------------------------------------------------------
Total: 151.20 pts | Average: 21.60 pts/game




## 🗑️ Database Management

### Clear Specific League Data


In [ ]:
def clear_league_data(league_id=None):
    """Clear data for a specific league."""
    if league_id is None:
        league_id = LEAGUE_ID
    
    if not league_id:
        print("❌ Please provide a league_id")
        return
    
    confirm = input(f"⚠️ Delete all data for league {league_id}? Type 'DELETE' to confirm: ")
    
    if confirm == "DELETE":
        with LeagueDB() as db:
            db.clear_league_data(league_id)
        print(f"\n✓ Cleared all data for league {league_id}")
    else:
        print("\n✗ Cancelled")

# Uncomment to run:
# clear_league_data()


### Clear Entire Database


In [ ]:
def clear_entire_database():
    """Delete all data from the database."""
    confirm = input("⚠️ Are you sure you want to delete ALL data? Type 'DELETE ALL' to confirm: ")
    
    if confirm == "DELETE ALL":
        with LeagueDB() as db:
            db.clear_all_data()
        print("\n✓ Deleted all data from database")
        print("Database is now empty.")
    else:
        print("\n✗ Cancelled. Database unchanged.")

# Uncomment to run:
# clear_entire_database()
# show_database_status()


## 🎯 Quick Reference

### Common Commands

```python
# View database status
show_database_status()

# Initial setup (run once)
find_my_leagues()  # Find your league IDs
# Update LEAGUE_ID in config cell
initial_data_load()  # Load all data

# Weekly updates
weekly_update()  # Update for current week
weekly_update(week=10)  # Update specific week

# View data
show_standings()  # League standings
show_matchups()  # Current week matchups
show_matchups(week=8)  # Specific week
show_team_roster(1)  # View a team's roster
show_bye_weeks()  # All bye weeks
show_bye_weeks(week=10)  # Specific week
show_player_stats("Player Name")  # Player lookup

# Database management
clear_league_data()  # Clear current league
clear_entire_database()  # Clear everything
```

### Database Tables

- `leagues` - League information and settings
- `users` - League owners/members
- `rosters` - Teams and their players
- `matchups` - Weekly game results
- `nfl_players` - Complete NFL player database
- `player_stats` - Weekly player performance
- `nfl_schedules` - Team schedules and bye weeks
- `transactions` - Trades, adds, drops

### Direct Database Access

For advanced queries:

```python
with LeagueDB() as db:
    # Your custom queries here
    rosters = db.get_rosters(LEAGUE_ID)
    players = db.get_nfl_players(position='RB')
    stats = db.get_player_stats(season='2024', week=9)
```

### Tips

1. Run `initial_data_load()` once to populate the database
2. Run `weekly_update()` every Monday/Tuesday for latest results
3. Check `show_database_status()` to see what data you have
4. Use `show_player_stats()` to look up any player
5. Check `show_bye_weeks()` before setting lineups

### Need More Help?

- **Complete Guide**: See `LEAGUE_DATA_GUIDE.md`
- **Quick Start**: See `QUICK_START_LEAGUE.md`
- **Example Script**: Run `example_league_usage.py`
